In [ ]:
import numpy as np
import pandas as pd
import torch
import pickle
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
data = pickle.load(open('bert_base_uncased/custom_bert_model.pkl', 'rb'))

model = data['model']
tokenizer = data['tokenizer']
label_encoder = data['label encoder']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def get_label_with_threshold(text):
    text = str(text)
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Ensure that input tensors are on the same device as the model
    input_ids = encoded_inputs['input_ids'].to(device)
    attention_mask = encoded_inputs['attention_mask'].to(device)

    # Forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Convert logits to probabilities and move to CPU for processing
    logits = logits.detach().cpu()
    class_probabilities = torch.softmax(logits, dim=-1).tolist()[0]

    class_names = np.arange(len(class_probabilities))
    class_probabilities_dict = dict(zip(class_names, class_probabilities))

    max_probability = max(class_probabilities_dict.values())
    predicted_class = max(class_probabilities_dict, key=class_probabilities_dict.get)

    return predicted_class, max_probability

In [ ]:
def get_label(arg):
  data = get_label_with_threshold(str(arg))
  class_name = label_encoder.inverse_transform([data[0]])[0]
  confidence = data[1]

  return (class_name, confidence)

In [ ]:
get_label_with_threshold('its a good day')

(2, 0.3897229731082916)

In [ ]:
get_label('its a good day')

('Negative', 0.3463916480541229)